In [1]:
import requests
import re
import pandas as pd
import xlrd
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile
import xml.etree.ElementTree as ET
import time
import FinanceDataReader as fdr
from bs4 import BeautifulSoup
import os
import numpy as np

In [2]:
crtfc_key = open('crtfc_key.txt',mode='r').readline()
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'

# Corpcode

## Corpcode.xml 가져오기

In [ ]:
url_corp = 'https://opendart.fss.or.kr/api/corpCode.xml?crtfc_key='+crtfc_key
# requests.get(url_corp).content.decode("UTF-8")
with urlopen(url_corp) as zipresp:
    with ZipFile(BytesIO(zipresp.read())) as zfile:
        zfile.extractall('Raw/Corpcode/corp_num')

## Corpcode.xml에서 기업코드 가져오기

In [ ]:
tree = ET.parse('Raw/Corpcode/corp_num/CORPCODE.xml')
root = tree.getroot()

corp_df = pd.DataFrame(columns=['corp_code','corp_name','stock_code','modify_date'])
for company in root.iter('list'):
    stock_code = company.findtext('stock_code')
    stock_code = stock_code.strip()
    if stock_code:
        company_dict = {
            'corp_code':company.findtext('corp_code'),
            'corp_name':company.findtext('corp_name'),
            'stock_code':company.findtext('stock_code'),
            'modify_date':company.findtext('modify_date')
        }
        corp_df = corp_df.append(company_dict, ignore_index=True)
# corp_df

# 필요 함수

## 함수생성을 위한 TEST

In [ ]:
corp_code = stock_code_reset(indexerror_list[0])
bgn_de = "19990101"
url_api = "https://opendart.fss.or.kr/api/list.xml?crtfc_key=" + crtfc_key + "&corp_code=" + corp_code + "&bgn_de=" + bgn_de +"&last_reprt_at=Y" + "&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003" + "&page_no=10&page_count=100"
dart_page = requests.get(url_api).content.decode("UTF-8")
rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', dart_page)
period_list = re.findall(r'<report_nm>(.*?)</report_nm>', dart_page)
year_list = list(map(year_reset, period_list))


In [ ]:
corp_df[corp_df['stock_code'] == '295820']

In [ ]:
rcp_no = '20050816000742'

In [ ]:
url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp)
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
resp = requests.get(url,headers={'user-agent':user_agent})
webpage = resp.content.decode("UTF-8")
webpage_clean = ['\n','\t',' ','(',')',"'",'"']
for clean in webpage_clean:
    webpage = webpage.replace(clean,'')
# try:
# webpage = re.findall(r"연결재무제표(.*?).xsd",webpage)[0]
# webpage = re.findall("부속명세서(.*?).xsd",webpage)[0]
# except:
# webpage = re.findall(r"재무제표(.*?).xsd",webpage)[0]
# webpage = re.findall(r"viewDoc(.*)",webpage)[0]
# dcm_eleid_no = webpage.split(',')[1:3]

In [ ]:
webpage

In [ ]:
eleid

In [ ]:
soup = BeautifulSoup(resp.content, 'html.parser')
soup

In [ ]:
# making_rcp_df(stock_code_reset('036670'))
making_rcp_df(stock_code_reset('005930'))

## 실행 함수

In [3]:
def making_rcp_df(corp_code):
    bgn_de = "19990101"
    end_de = "20210410"
#     bgn_de = "20200101"
    url_api = "https://opendart.fss.or.kr/api/list.xml?crtfc_key=" + crtfc_key + "&corp_code=" + corp_code + "&bgn_de=" + bgn_de +"&end_de="+end_de+"&last_reprt_at=Y" + "&pblntf_detail_ty=A001&pblntf_detail_ty=A002&pblntf_detail_ty=A003" + "&page_no=10&page_count=100"
    dart_page = requests.get(url_api).content.decode("UTF-8")
    rcp_no_list = re.findall(r'<rcept_no>(.*?)</rcept_no>', dart_page)
    period_list = re.findall(r'<report_nm>(.*?)</report_nm>', dart_page)
    year_list = list(map(year_reset, period_list))
    
#     dcm_no_list = list(map(making_dcm, rcp_no_list))
    dcm_eleid_no = list(map(making_dcm_eleid,rcp_no_list))
    dcm_no_list = pd.DataFrame(dcm_eleid_no)[0].to_list()
    eleid_list = pd.DataFrame(dcm_eleid_no)[1].to_list()
    
    stock_code = corp_code_reset(corp_code)
    
    rcp_no_list.insert(0,stock_code)
    rcp_no_list.insert(1,'rcp_no')
    year_list.insert(0,'stock_code')
    year_list.insert(1,'description')
    dcm_no_list.insert(0,stock_code)
    dcm_no_list.insert(1,'dcm_no')
    eleid_list.insert(0,stock_code)
    eleid_list.insert(1,'eleid')
    
#     rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list],columns=year_list)
    rcp_df = pd.DataFrame(data=[rcp_no_list,dcm_no_list,eleid_list],columns=year_list)
    rcp_df = rcp_df.loc[:, ~rcp_df.columns.duplicated(
        keep='first')]  #한 해에 여러개의 보고서가 나오면 최근거만 가져옴
    return rcp_df

# def making_dcm (rcp_no):
#     url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
#     user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
#     resp = requests.get(url,headers={'user-agent':user_agent})
#     webpage = resp.content.decode("UTF-8")
#     dcm_no = re.findall(r"{}', '(.*?)',".format(rcp_no),webpage)[0]
#     time.sleep(0.5)
#     return dcm_no

def making_dcm_eleid (rcp_no):
#     print(rcp_no)
    url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp_no)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp = requests.get(url,headers={'user-agent':user_agent})
    webpage = resp.content.decode("UTF-8")
    webpage_clean = ['\n','\t',' ','(',')',"'",'"']
    for clean in webpage_clean:
        webpage = webpage.replace(clean,'')
    search_words = ['연결재무제표','재무제표','부속명세서']
    for search_word in search_words:
        try :
            webpage = re.findall(search_word+"(.*?).dart",webpage)[0]
            break
        except:
            continue
    webpage = re.findall(r"viewDoc(.*)",webpage)[0]
    dcm_eleid_no = webpage.split(',')[1:3]
    time.sleep(0.5)
    return dcm_eleid_no

def corp_code_reset(corp_code):  #에러가 발생할 경우 회사 코드 불러오기 실핸한다.
    stock_code = corp_df[corp_df['corp_code'] == corp_code].iloc[0, 2]
    return stock_code

def stock_code_reset(stock_code):
    corp_code = corp_df[corp_df['stock_code'] == stock_code].iloc[0, 0]
    return corp_code

def year_reset(title):
    try:
        year = title[-8:-1] +'_'+ title[-15:-10]
    except:
        year = '0'
    return year


# rcp, dcm 가져오기

## stock_code 가져오기

In [4]:
stock_list_kospi = fdr.StockListing('KOSPI')['Symbol'].to_list()
stock_list_kosdaq = fdr.StockListing('KOSDAQ')['Symbol'].to_list()
stock_df_del = fdr.StockListing('KRX-DELISTING')
stock_list_del = stock_df_del[(stock_df_del['Market'] == "KOSPI") | (stock_df_del['Market'] == "KOSDAQ")]['Symbol'].to_list()

In [ ]:
# stock_code_list = ['005930','005380','017670']

In [5]:
# stock_code_list 정제하기 (우선주, 리츠, 옵션) -> 6자리 숫자만, 맨 앞자리에 0 들어가는 것만
stock_code_list = []
for stock_code in stock_list_kospi:
    stock_code = re.findall("\d+", stock_code)[0] # 숫자만 추출한 뒤 첫번째 것만 사용
    if (stock_code[-1] == '0') and (len(stock_code) == 6):
        stock_code_list.append(stock_code)

## rcp, dcm, eleid 만들기

In [ ]:
# start = time.time()
rcp_df = pd.DataFrame()
indexerror_list=[]
valueerror_list=[]
start_time = time.time()
i=0
for stock_code in stock_code_list:
#     time_gap = time.time() - start_time
#     if time_gap > 60:
#         print('Runtime > 60sec')
#         break
    i=i+1
    if i % 10 == 0:
        print(i)
        rcp_df.reset_index(drop=True,inplace=True)
        rcp_df.to_csv('Raw/rcp_dcm/rcp_df.csv', index=True, encoding='utf-8-sig')
    try :
        start_time = time.time()
        corp_code = stock_code_reset(stock_code)
        rcp_df_temp = making_rcp_df(corp_code)
        rcp_df = pd.concat([rcp_df,rcp_df_temp],axis=0)
    except IndexError:
        indexerror_list.append(stock_code)
    except ValueError:
        valueerror_list.append(stock_code)
    except Exception as e:
        print (stock_code,e)
    
rcp_df.reset_index(drop=True,inplace=True)
# print(time.time()-start)
rcp_df.to_csv('Raw/rcp_dcm/rcp_df.csv', index=True, encoding='utf-8-sig')
# IndexError : single positional indexer is out-of-bounds : 우선주 등 재무제표 없는 경우
# list index out of range : 보고서 제목에 년도가 안써있는 경우 -> 0으로 쓰는것으로 수정
# ValueError : Length mismatch: Expected axis has 0 elements, new values have 1 elements : 리츠 등 재무제표 없는 경우

In [ ]:
rcp_df

In [ ]:
rcp_df
indexerror_list
making_rcp_df(stock_code_reset(indexerror_list[0]))

## rcp, dcm 불러오기

In [ ]:
def six_degree (stock_code):
    stock_code_return = '%06d'%(stock_code)
    return stock_code_return

In [ ]:
rcp_df = pd.read_csv('Raw/rcp_dcm/rcp_df.csv')
# rcp_df = rcp_df.set_index(rcp_df.columns[0])
# rcp_df.index.name = None
rcp_df = rcp_df.drop(columns=rcp_df.columns[0])
rcp_df.reset_index(drop=True,inplace=True)
# column 지정
# rcp_df.columns = [c.columns.map(lambda x : x[:2]), c.iloc[0], c.iloc[1]]
# c = c.iloc[2:]
# 종목명 6자리로 변경
# for i in range(len(rcp_df.index)):
#     rcp_df.rename(index={rcp_df.index[i]:'%06d'%(rcp_df.index[i])}, inplace = True)
rcp_df['stock_code']=list(map(six_degree, rcp_df['stock_code']))

# rcp_df

# 재무제표 다운로드

## Error_case
- 연결재무제표가 제출예정인 경우, rcp = 20070330001989 (005930, 2006.12), 20060331001832 (005930, 2005.12),('005930', '2004.12'),(005380, 2006.12),(005380, 2005.12),(005380, 2004.12),(005380, 2002.12),(005380, 2001.12),(005380, 2000.12),(005380, 1998.12)
~> 연결 감사보고서로 검색
- 연결재무제표에 '재무상태표'등 항목명이 안써있는 경우 : (005930, 2005.09), (005930, 2005.06), (005930, 2004.09),(005930, 2004.06)
~> 찾는 방법 추가
- 2000년대 초반에는 반기, 분기 보고서의 연결재무제표에서는 분기자료가 아닌 연간 자료가 입력되어 있다. (현대자동차로 보니 2009년 부터 분기 자료로 들어가 있는듯)
~> 상단의 기간 테이블을 같이 추출하기 or 일단 최근 10년의 데이터만으로 시작해보기.(오류항목 더 적음, 다운로드만으로 가능할듯)

In [ ]:
stock_code, year

In [ ]:
# error_df.to_csv('error_df.csv')
# error_df=pd.read_csv('error_df.csv',encoding='utf-8',index_col=0).astype(int)
error_df

In [ ]:
error_df = pd.DataFrame(index = ['stock_code','year','rcp','dcm','eleid'])

for no_of_stock in range(int(len(rcp_df.index)/3)):
    rcp_list_temp = rcp_df.iloc[3*no_of_stock,2:].values
    dcm_list_temp = rcp_df.iloc[3*no_of_stock+1,2:].values
    eleid_list_temp = rcp_df.iloc[3*no_of_stock+2,2:].values
    year_list_temp = rcp_df.columns[2:]
    stock_code_temp = rcp_df.iloc[3*no_of_stock,0]
    for year, rcp, dcm, eleid in zip(year_list_temp,rcp_list_temp,dcm_list_temp,eleid_list_temp):
        try:
            download_ftable_html(stock_code_temp,year,rcp,dcm,eleid)
        except:
            no_error=len(error_df.columns)
            error_df[no_error] = [stock_code_temp,year,rcp,dcm,eleid]

In [6]:
# 2007.09 부터 가능
def download_ftable_excel(stock_code,year,rcp,dcm):
    url_download = 'http://dart.fss.or.kr/pdf/download/excel.do?rcp_no={}&dcm_no={}&lang=ko'.format(rcp,dcm)
    user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    resp_download = requests.get(url_download, headers = {'user-agent' : user_agent}, verify=False)
    table_download = BytesIO(resp_download.content)
    sheet_names = pd.ExcelFile(table_download).sheet_names
    sheets_group = [['연결 재무상태표','재무상태표','대차대조표'],['연결 현금흐름표','현금흐름표','현금흐름표'],['연결 포괄손익계산서','포괄손익계산서','손익계산서']]
    for sheets in sheets_group:
        i=0
        while True:
            if sheets[i] not in sheet_names:
                i=i+1
                if i > len(sheets):
                    print("재무상태표를 찾을 수 없습니다")
                    break
            else: 
                data_download = pd.read_excel(table_download, sheet_name = sheets[i])
                try:
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='euc-kr')
                except UnicodeEncodeError:
                    # 최근것은 euc-kr로 encoding되어 있으나, 2010년이전에는 utf-16으로 encoding된것으로 보임.
                    # 다만, utf-16에서 to_csv할 때 sep=','이 적용되지 않아 다운로드된 파일의 글씨는 깨지지만 utf-8로 사용.
                    data_download.to_csv("Raw/Dart_Crawling/"+stock_code+"_"+year+"_"+sheets[1]+".csv", encoding='utf-8')
                break

In [ ]:
def download_ftable_html(stock_code,year,rcp,dcm,eleid):
    url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo={}&dcmNo={}&eleId={}&offset=687340&length=115750&dtd=dart3.xsd'.format(int(rcp),int(dcm),int(eleid))
    table = pd.read_html(url)
    sheets_group = [['연결재무상태표','재무상태표','대차대조표'],['연결현금흐름표','현금흐름표','현금흐름표'],['연결포괄손익계산서','포괄손익계산서','손익계산서']]
    for sheets in sheets_group:
        for sheet in sheets:
            find=0
            for i in range(len(table)):
                if sheet in str(table[i].iloc[0,0]).replace(' ',''):
                    unit = table[i].iloc[-1,0]
                    table_temp = table[i+1]
                    table_temp['unit'] = unit
                    table_temp.to_csv("Raw/Dart_Crwaling_Table/"+str(stock_code)+"_"+str(year)+"_"+sheets[1]+".csv", encoding='utf-8-sig')
                    find=1
                    break
            if find == 1:
                break


In [ ]:
stock_code='005380'
year='1998.12'
rcp,dcm,eleid = rcp_df[rcp_df['stock_code']==stock_code][year].values

In [ ]:
stock_code, year,rcp, dcm, eleid = error_df.iloc[:,0].values
# error_df.iloc[:,0].values

In [ ]:
download_ftable_html(stock_code, year,rcp, dcm, eleid)

In [ ]:
import webbrowser
url = 'http://dart.fss.or.kr/dsaf001/main.do?rcpNo={}'.format(rcp)
webbrowser.open(url)

In [ ]:
url = 'http://dart.fss.or.kr/report/viewer.do?rcpNo={}&dcmNo={}&eleId={}&offset=687340&length=115750&dtd=dart3.xsd'.format(int(rcp),int(dcm),int(eleid))
table = pd.read_html(url)

In [ ]:
# for i in range(len(table)):
#     print(table[i].iloc[0,0])
# str(table[1].iloc[0,0]).replace(' ','')
table[1]

# Download CSV 활용

설명서
- 필요 항목명 정리 : 각 재무제표 마다 다를 것.
1. 필요한 자료
* 재무상태표 : 지배주주순자산(PBR), 총자산, 증자여부, 현금(비영업자산), 부채
* 손익계산서 : 매출액(PSR), 매출원가(매출총이익)(GP/A), 판관비, 영업이익(EBIT),  당기순이익, 지배주주순이익(PER)
* 현금흐름표 : 영업활동현금흐름
- [['매출액','수익매출액'],['매출총이익'],['판매비와관리비'],['영업이익손실'],['당기손이익손실'],['지배기업의소유주에게귀속되는당기순이익손실']]

2. 항목을 가져오는 다양한 아이디어 고민 
- 같은 종목이라면 항목명을 동일하게 가져갔을 것이라고 가정하여 종목별 항목명 자료 구성
- 다음 년도에 같은 값을 가진 항목을 불러오기 등

3. 분기 단위 값을 연 단위로 처리하기
- database.ipynb 에 만들어 놓은 함수 사용
4. 가격 데이터 or 시가총액 데이터 가져오기
- pykrx 사용

## CSV Cleaning

In [ ]:
data_path = './Raw/Dart_Crawling/'
files_list = os.listdir(data_path)

In [ ]:
file_temp = files_list[0]
file = data_path + file_temp
reading_csv(file)

In [ ]:
for file_temp in files_list:
    print(file_temp)
    file = data_path + file_temp
    csv_filtered = reading_csv(file)
    file_filtered = file.replace('Dart_Crawling','Dart_Crawling (Filtered)')
    try:
        csv_filtered.to_csv(file_filtered,encoding ='cp949')
    except UnicodeEncodeError :
        csv_filtered.to_csv(file_filtered,encoding ='utf-8')
        

In [ ]:
csv_temp = pd.read_csv(file, encoding ='utf-8', index_col = 0)
for i in range(len(csv_temp.index)):
    if '단위' in str(csv_temp.iat[i,0]):
        slice_point = i
        break
unit_korean = csv_temp.iat[slice_point,0]
if '십억원' in unit_korean:
    unit = 10**9 / 10**8
elif'백만원' in unit_korean: 
    unit = 10**6 / 10**8
elif '천원' in unit_korean:
    unit = 10**3 / 10**8
else:
    print(file + '_check unit_korean')
csv_temp_filtered = csv_temp.iloc[slice_point+1:,:]
csv_temp_filtered = csv_temp_filtered.dropna(how='all')
csv_temp_filtered.columns = csv_temp_filtered.iloc[0,:]
csv_temp_filtered.index = csv_temp_filtered.iloc[:,0]
csv_temp_filtered = csv_temp_filtered.iloc[1:,1:]
csv_temp_filtered.columns.name=''


In [ ]:
def reading_csv (file):
    try:
        csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0) # index_col : index로 삼을 열 지정
#         print('cp949')
    except UnicodeDecodeError:
        csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
#         print('utf-8')
    # 단위가 적혀 있는 cell찾기
    for i in range(len(csv_temp.index)):
        if '단위' in str(csv_temp.iat[i,0]):
            slice_point = i
            break
    # 원 단위로 하면 보기가 불편하여 억 단위로 반영
    unit_korean = csv_temp.iat[slice_point,0]
    if '십억원' in unit_korean:
        unit = 10**9 / 10**8
    elif'백만원' in unit_korean: 
        unit = 10**6 / 10**8
    elif '천원' in unit_korean:
        unit = 10**3 / 10**8
    else:
        print(file + '_check unit_korean')
    # 재무제표 자료만을 수정
    csv_temp_filtered = csv_temp.iloc[slice_point+1:,:]
    csv_temp_filtered = csv_temp_filtered.dropna(how='all')
    csv_temp_filtered.columns = csv_temp_filtered.iloc[0,:]
    csv_temp_filtered.index = csv_temp_filtered.iloc[:,0]
    csv_temp_filtered = csv_temp_filtered.iloc[1:,1:]
    csv_temp_filtered.columns.name=''
    csv_temp_filtered = csv_temp_filtered.apply(pd.to_numeric,errors = 'coerce') #text가 있어서 float를 변환이 안될 경우에 NaN으로 변환
#     csv_temp_filtered = csv_temp_filtered.replace('\xa0',np.nan) # \xa0는 줄바꿈방지 공백(Non-breaking Space,?)로 결측치로 수정
#     csv_temp_filtered = csv_temp_filtered.replace(' ',np.nan) # 값이 없는 경우 ' '으로 되어 있어 결측치로 수정
    csv_temp_filtered = csv_temp_filtered.dropna(how='all') # 결측치를 삭제
#     csv_temp_filtered = csv_temp_filtered.astype(float) # 값 형식이 str으로 되어 있어 float로 수정
    csv_temp_filtered = csv_temp_filtered * unit
    # index를 clearing
    index = list(map(lambda x : (''.join(re.compile('[가-힣]+').findall(str(x)))).replace('주석','').rstrip('주'), csv_temp_filtered.index))
    csv_temp_filtered.index = index
    return csv_temp_filtered

## CSV 원하는 항목 추출

- 필요한 자료
- 재무상태표 : 지배주주순자산(PBR), 총자산, 증자여부, 현금(비영업자산), 부채
- 재무상태표(Base) : 지배주주순자산 = 자산총계 - 비지배지분

- 손익계산서 : 매출액(PSR), 매출원가(매출총이익)(GP/A), 판관비, 영업이익(EBIT), 당기순이익, 지배주주순이익(PER)
- 현금흐름표 : 영업활동현금흐름
- [['매출액','수익매출액'],['매출총이익'],['판매비와관리비'],['영업이익손실'],['당기손이익손실'],['지배기업의소유주에게귀속되는당기순이익손실']]

In [ ]:
data_path = './Raw/Dart_Crawling (Filtered)/'
files_list = os.listdir(data_path)

In [ ]:
files_list_bs = []
files_list_is = []
files_list_cf = []
for file in files_list:
    if '재무상태표' in file:
        files_list_bs.append(file)
    if '손익계산서' in file:
        files_list_is.append(file)
    if '현금흐름표' in file:
        files_list_cf.append(file)

In [ ]:
stocks = []
for files in files_list:
    stock = files[0:6]
    stocks.append(stock)
stocks = list(set(stocks))

In [ ]:
file = data_path+files_list_bs[99]
file

In [ ]:
try:
    csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
except UnicodeDecodeError:
    csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
csv_temp

In [ ]:
stock = stocks[0]
items = ['자본총계','비지배지분']
csv_bs_temp = pd.DataFrame()
csv_bs_temp['stock'] = [stock]*len(items)
csv_bs_temp['item'] = items
csv_bs_temp

In [ ]:
for file in files_list_bs:
    if stock in file:
        file = data_path+file
        try:
            csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
        except UnicodeDecodeError:
            csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
        year = file[-17:-10]
        values_temp=[]
        for item in items:
            try:
                value_temp = csv_temp.at[item,csv_temp.columns[0]]
            except:
                value_temp = np.nan
            values_temp.append(value_temp)
        csv_bs_temp[year]=values_temp
csv_bs_temp

In [ ]:
for file in files_list_bs:
    if stock in file:
        try:
            csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
        except UnicodeDecodeError:
            csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
        value_temp = csv_temp.at['자본총계',csv_temp.columns[0]]
        csv_bs_temp = 
csv_bs_temp = 

In [ ]:
pd.DataFrame([['a',1,2,3]],index=['stock_code'],columns=['항목','year1','year2','year3'])

In [ ]:
csv_cf_all = pd.DataFrame()
for file in files_list_cf:
    file = data_path+file
    try:
        csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
    except UnicodeDecodeError:
        csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
    csv_temp = csv_temp.iloc[0:1,:] # iloc[0,:]형식으로 가져오면 series 형태로 가져온다.
    print(csv_temp.isnull().sum().sum())
    csv_cf_all = pd.concat([csv_cf_all,csv_temp],axis=1)

In [ ]:
csv_cf_all = pd.DataFrame(index = ['영업활동으로인한현금흐름'])
for file in files_list_cf:
    file = data_path+file
    try:
        csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
    except UnicodeDecodeError:
        csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
    year = file[-17:-10]
    value = csv_temp.iat[0,0] # iloc[0,:]형식으로 가져오면 series 형태로 가져온다.
    csv_cf_all[year] = value

In [ ]:
year = file[-17:-10]

In [ ]:
csv_cf_all

In [ ]:
file = data_path+files_list_cf[0]
try:
    csv_temp = pd.read_csv(file, encoding ='cp949',index_col = 0)
except UnicodeDecodeError:
    csv_temp = pd.read_csv(file, encoding ='utf-8',index_col = 0)
csv_temp.iloc[0,:]

# 가격 데이터 

In [ ]:
from pykrx import stock

In [ ]:
df = stock.get_market_cap_by_ticker("20200625")
print(df.head())

In [ ]:
df = stock.get_market_cap_by_date("20190101", "20190131", '152100').iloc[:,0:1]

print(df.head())

In [ ]:
df_cap = pd.DataFrame()
for stock_code in stock_code_list[0:50]:
    df_cap_temp = stock.get_market_cap_by_date("19990101", "20210101",stock_code).iloc[:,0:1]
    df_cap_temp.rename(columns = {'시가총액':stock_code},inplace=True)
    df_cap = pd.concat([df_cap,df_cap_temp],axis=1)

In [ ]:
df_cap

In [ ]:
# df_cap
stock_code_list

# ETC

In [ ]:
# i=4
for i in range(10):
    while True:
        if i != 6:
            if i == 8:
                print('break')
                break
            i=i+1
        else:
            print(i)
            break